# __Graph Game__

#### A game of chance... <br>

created by Artem, Femi, Tom, Zifan <br>
Github repo: __https://github.com/T0mLam/CS_Summative_Project__
***

# Quickstart Guide

## 1. Download all dependencies

- matplotlib
- networkx
- numpy
- pygame
- tkinter

In [ ]:
%pip install matplotlib
%pip install networkx
%pip install numpy
%pip install pygame
%pip install tk

## 2. Create all modules

In [4]:
from __future__ import annotations
from typing import List, Tuple, Dict

### Utilities modules 
These modules contain data structures required for the game.

In [5]:
import random as rand

#### Node Class
An individual node of the graph data structure.

In [6]:
class Node:
    """Creation of a Node class to represent nodes in the graph data structure."""

    def __init__(self, idx: int) -> None:
        """Initializes a Node object with the parameter value.

        Args:
            idx (int): This are the values associated with the node.
        """
        self.idx = idx
        self.neighbours = {}  # A dictionary to keep the neighbours and their weight

    def __lt__(self, other: type[Node]) -> bool:
        """Compares two nodes based on their values.

        Args:
            other (Node): Another node in the graph to compare with.

        Returns:
            True if the value of the other node is greater than the value of the this node, False otherwise.
        """
        return self.idx < other.idx

    def add_neighbour(self, neighbour: type[Node], weight: int | float) -> None: 
        """Adds a neighbour to the nodes in the graph with an associated weight.

        Args:
            neighbour (Node): Neighbour node to be added.
            weight (int, float): the Weight of the edge connecting the neighbour to the node.
            
        Raises:
            TypeError: If neighbour is not an instance of Node in the graph or if the weight inputed is not numeric (eg" writing a word instead of a number").
        """
        if not isinstance(neighbour, Node):
            raise TypeError("Neighbour must be an instance of Node.")
        if not isinstance(weight, (int, float)):
            raise TypeError("Weight must be a numeric value.")
        
        self.neighbours[neighbour] = weight

    def get_index(self) -> int:
        """A getter method for the idx attribute.

        Returns:
            An integer index of the node.
        """
        return self.idx
    
    def get_neighbours(self) -> List[Tuple[int, int]]:
        """A getter method for the neighbours attribute.

        Returns:
            A List of tuples (idx, wei) consist of the index (idx) and the weight (wei) of each node.
        """
        return self.neighbours.items()

#### Heap Class
A minheap data structure which enables the quick extraction of the minimum element. <br>
It is utilized in the shortest path method and the fizzy search method.

In [16]:
class MinHeap:
    """A minheap data structure that enables quick extraction of the minimum element.

    Time Complexity:
        Initialize / heapify O(n * log n)
        Insertion O(log n)
        Extract min O(log n)
        Peak min O(1)
    
    Methods:
        push: Push an element into the minheap.
        pop: Remove and return the smallest element from the minheap.
        top: Return the smallest element from the minheap without removing.
    """

    def __init__(self, arr: List[Tuple[int, type[Node]]] | None = None) -> None:
        """Construct a heap by heapifying the input array.

        Args:
            arr: A list of tuple to be heapified during the initializing stage.
                 For example: [(1, Node_Obj1), (2, Node_Obj2), ...]

        Raises:
            TypeError: An error due to the invalid input data type of arr.
        """
        # Check whether the input array is a list 
        if arr and not isinstance(arr, list):
            raise TypeError("The parameter 'arr' must be a python list")

        self.__heap = arr if arr else []
        self.__heapify()

    def __len__(self) -> int:
        """Return the number elements in the minheap.

        Returns:
            An integer representing the number of element in the current minheap.
            For example: 

            heap = MinHeap([1, 2, 3, 4, 5]) 
            print(len(heap))
            # Output: 5
        """
        return len(self.__heap)
    
    def push(self, val: Tuple[int, type[Node]]) -> None:
        """Push the value into MinHeap.

        Args:
            val: Tuple with the integer and a node that will be pushed to the heap.
        """
        #The current index where the new value will be put 
        curr = len(self.__heap)
        #Append the new value to the heap
        self.__heap.append(val)
        #Using the __get_parent_idx to get parent index
        parent = self.__get_parent_idx(curr)
        
        #Compare the new element with its parent to check if it is not bigger that its parent and swap if it is
        while self.__heap[curr] < self.__heap[parent]:
            self.__swap(curr, parent)
            curr = parent
            parent = self.__get_parent_idx(parent)
        
    def pop(self) -> Tuple[int, type[Node]]:
        """Returns and deletes the smallest element from the heap.

        Returns:
            A tuple consist of the an integer and a node object or None if the heap is empty.
            For example:

            heap = MinHeap([(1, Node_Obj1), (2, Node_Obj2), ...])
            print(heap.pop())
            # Output: (1, Node_Obj1)
        """
        if not self.__heap:
            return
        #Change the root noed for th last node using the __swap method
        self.__swap(0, len(self.__heap) - 1)
        #Creating the variable for the smalelst item
        min_element = self.__heap.pop()
        #Using __sift down method for keeping the heap property
        self.__sift_down(0)
        #return the element
        return min_element

    def top(self) -> Tuple[int, type[Node]]:
        """Return the smallest element from the heap.

        Returns:
            A tuple consist of the an integer and a node object or None if the heap is empty.
            For example:

            heap = MinHeap([(1, Node_Obj1), (2, Node_Obj2), ...])
            print(heap.top())
            # Output: (1, Node_Obj1)
        """
        if not self.__heap:
            return
        return self.__heap[0]

    def __heapify(self) -> None:
        """Heapify the input array by sifting down all the non-leaf nodes."""
        for idx in range(len(self.__heap) // 2, -1, -1):
            self.__sift_down(idx)

    def __sift_down(self, idx: int) -> None:
        """A heap operation thats maintain the min heap structure.

        It allocates the nodes with smallest values at the root and largest values at the leaves
        by sifting down every nodes in the wrong positions.
        """
        left_child_idx, right_child_idx = self.__get_left_child_idx(idx), self.__get_right_child_idx(idx)

        # Check whether the current node has 2 children
        if min(left_child_idx, right_child_idx) != -1:
            # Check whether the current node is smaller than both its children
            if min(self.__heap[idx],
                   self.__heap[left_child_idx],
                   self.__heap[right_child_idx]) != self.__heap[idx]:
                # If the left child is smaller than the right child,
                # swap the left child with the current node and sift down the left child
                if self.__heap[left_child_idx] < self.__heap[right_child_idx]:
                    self.__swap(idx, left_child_idx)
                    self.__sift_down(left_child_idx)
                # Otherwise, swap the right child with the current node and sift down the right child
                else:
                    self.__swap(idx, right_child_idx)
                    self.__sift_down(right_child_idx)
            
        # Else if the current node is larger than its only left child,
        # swap the left child with the current node and sift down the left child
        elif (left_child_idx != -1 and
              self.__heap[left_child_idx] < self.__heap[idx]):
            self.__swap(idx, left_child_idx)
            self.__sift_down(left_child_idx)
    
        # Else if the current node is larger than its only right child,
        # swap the right child with the current node and sift down the right child
        elif (right_child_idx != -1 and
              self.__heap[right_child_idx] < self.__heap[idx]):
            self.__swap(idx, right_child_idx)
            self.__sift_down(right_child_idx)
     
    def __swap(self, idx1: int, idx2: int) -> None:
        """Swap the elements with index idx1 and idx2 in the heap."""
        self.__heap[idx1], self.__heap[idx2] = self.__heap[idx2], self.__heap[idx1]

    def __get_parent_idx(self, idx: int) -> int:
        """Get the parent of the element with index idx.

        Returns:
            An integer representing the index of the parent, or -1 if no parent is found.
        """
        return idx // 2
    
    def __get_left_child_idx(self, idx: int) -> int:
        """Get the left child of the element with index idx.

        Returns:
            An integer representing the index of the left child, or -1 if no left child is found.
        """
        left_child_idx = 2 * idx + 1
        return left_child_idx if left_child_idx < len(self.__heap) else -1

    def __get_right_child_idx(self, idx: int) -> int:
        """Get the right child of the element with index idx.

        Returns:
            An integer representing the index of the right child, or -1 if no right child is found. 
        """
        right_child_idx = 2 * idx + 2
        return right_child_idx if right_child_idx < len(self.__heap) else -1

#### RandomisedSet Class
A set data structure which enables O(1) (constant time) insertion, removal and random extraction of elements at the expense of additional memory. <br>
It is used for storing the unconnected edges for the random graph generation in the Graph class.

In [ ]:
class RandomisedSet:
    """A randomised set which enables the insertion, deletion and random extraction of edges in O(1) (constant) time.

    Attributes:
        edges: A list storing non-duplicate edges that are not connected in the graph.
        edge_to_idx: A hashmap mapping the edges to the indices they are stored in the edges list.

    Methods:
        add_edges_from_node: Add available edges connected to all nodes from the input node.
        add_edge_to_set: Add an edge to the randomised set.
        remove_edge_from_set: Remove an edge from the randomised set.
        get_random_edge: Extract a random available edge.
    """

    def __init__(self) -> None:
        """Construct the attributes of the set."""
        self.edges = []
        self.edge_to_idx = {}

    def __contains__(self, item: Tuple[int, int]) -> bool:
        """Enable the use of membership test operators (in & not in) for the class.

        Args:
            item (tuple): A tuple to be checked whether it is contained in the edges set.

        Returns:
            True if the edge is found, false otherwise.
        """
        idx1, idx2 = item
        return (idx1, idx2) in self.edge_to_idx or (idx2, idx1) in self.edge_to_idx

    def add_edges_from_node(self, idx: int, nodes: List[int]) -> None:
        """Add outcoming edges to the set from a new node.
        
        For example:
        idx: 5
        nodes: [1, 3, 7, 8]
        # New edges created: (1, 5), (3, 5), (5, 7), (5, 8)

        Notes: 
            Adding n edges in O(n) time, where n = num of input nodes.
            Amortized time complexity for adding an edge: O(1).

        Args:
            idx (int): The index of the new node.
            nodes (list): The indices of the existing nodes.

        Raises:
            TypeError: Errors caused by incompatible input data type of 'idx' and 'nodes'.
        """
        # Check data types of parameters 'idx' and 'nodes'
        if not isinstance(idx, int):
            raise TypeError("The input parameter 'idx' must be an integer")
        if (not isinstance(nodes, list) or 
            not all([isinstance(node, int) for node in nodes])):
            raise TypeError("The input parameter 'nodes' must be a list of integers")

        # Add the new edges to the set
        for node_idx in nodes:
            # Skip to the next iteration if the index of starting node = ending node
            if node_idx == idx:
                continue
            # Format the new edge as (u, v), where u < v
            mn, mx = min(idx, node_idx), max(idx, node_idx)
            new_edge = (mn, mx)

            # Check whether the new edge exists
            if new_edge not in self.edge_to_idx:
                # If not, add it to the list and map the edge to its indices in the list
                self.edge_to_idx[new_edge] = len(self.edges)
                self.edges.append(new_edge)

    def add_edge_to_set(self, idx1: int, idx2: int) -> None:
        """Add an edge to the randomised set.

        Args:
            idx1 (int): Index of the first node.
            idx2 (int): Index of the second node.

        Raises:
            TypeError: Error occurs if the node indices are not integers.
            ValueError: Error occurs if the node indices are out of bounds.
        """
        # Check if indices are integers
        if not isinstance(idx1, int) or not isinstance(idx2, int):
            raise TypeError("Node indices must be integers.")

        # Check if indices are greater than 0
        if idx1 < 1 or idx2 < 1:
            raise ValueError("Node indices must be positive.")
        
        # making sure idx1 is less than idx2 to maintain consistency
        idx1, idx2 = min(idx1, idx2), max(idx1, idx2)
        new_edge = (idx1, idx2)

        # Add the new edge to the randomised set 
        if new_edge not in self.edge_to_idx:
            self.edge_to_idx[new_edge] = len(self.edges)
            self.edges.append(new_edge)

    def remove_edge_from_set(self, idx1: int, idx2: int) -> None:
        """Remove an edge from the set.

        Notes:
            First swap the target edge with the last edge in the edges list and pop it out of the list.
            
        Args:
            idx1 (int): Index of the first node.
            idx2 (int): Index of the second node.

        Raises:
            TypeError: Error occurs if the node indices are not integers.
            ValueError: Error occurs if the edge does not exist.
        """
        # Check if indices are integers
        if not isinstance(idx1, int) or not isinstance(idx2, int):
            raise TypeError("Node indices must be integers.")

        # Check if indices exist in the set
        removing_edge = (min(idx1, idx2), max(idx1, idx2))
        if removing_edge not in self.edge_to_idx:
            raise ValueError("The edge does not exist in the set") 
        
        # Get the indices of the edge to be removed and the last edge in the list
        removing_edge_index = self.edge_to_idx[removing_edge]
        last_edge = self.edges[-1]

        # Replace the edge to be removed with the last edges and change the corresponding index in the node map
        self.edge_to_idx[last_edge] = removing_edge_index
        self.edges[removing_edge_index] = last_edge

        # Remove the last entry in the edges list and delete the index map for the removed edge 
        self.edges.pop()
        del self.edge_to_idx[removing_edge]

    def get_random_edge(self) -> Tuple[int, int]:
        """Extract and remove a random available edge from the set.

        Returns:
            A tuple consists of the start and end of the edge. For example: (1, 2).
        """
        if not self.edges:
            return
        
        # Randomly select an edge from the list
        edge_idx = rand.randint(0, len(self.edges) - 1)
        edge = self.edges[edge_idx]

        # Delete the edge from the set and return it
        self.remove_edge_from_set(*edge)
        return edge

#### Trie Class

#### Graph Class

### Additional Features Modules
These modules contain the database for storing player data and the leaderboard search engine for searching players.

In [ ]:
import sqlite3
import os

#### DatabaseConnection Class
A context manager class which automatically handles the connection and disconnection of the database.

In [ ]:
class DatabaseConnection:
    """A context manager class for handling database connections."""
    
    def __init__(self, db_name):
        """Initialize the DatabaseConnection object."""
        self.db_name = db_name
        self.connection = None

    def __enter__(self):
        """Enter method for the context manager."""
        try:
            self.connection = sqlite3.connect(os.path.dirname(__file__) + '/' + self.db_name)
            return self.connection
        except sqlite3.Error as e:
            print(f"Error connecting to database: {e}")
            raise
       
    def __exit__(self, exc_type, exc_val, exc_tb):
        """Exit method for the context manager."""
        if self.connection:
            try:
                self.connection.commit()
            except sqlite3.Error as e:
                print(f"Error committing changes to database: {e}")
            finally:
                self.connection.close()

#### Authentication Modules

#### SearchEngine Class

### Game Logic Modules
These modules contain the game logic and the user interface of the game.

#### RandomScoreGenerator Class

#### GraphGame Class

#### GraphGameGUI Class

In [ ]:
class GraphGameGUI:
    pass

## 3. Run all the unit tests
Unit tests are conducted using the unittest module 

In [20]:
import unittest
from unittest.mock import patch

### Testing data structures

#### Test Node class

In [ ]:
class TestNode(unittest.TestCase):
    def setUp(self):
        self.node1 = Node(1)
        self.node2 = Node(2)
        self.node3 = Node(3)

    def test_node_creation(self):
        """Tests whether the node is created correctly"""
        self.assertEqual(self.node1.idx, 1)
        self.assertEqual(self.node2.idx, 2)

    def test_add_edge(self):
        """Testing the neighbor addition function"""
        self.node1.add_neighbour(self.node2, 10)
        self.assertIn(self.node2, self.node1.neighbours)
        self.assertEqual(self.node1.neighbours[self.node2], 10)

        with self.assertRaises(TypeError):
            self.node1.add_neighbour("not_a_node", 5)
        with self.assertRaises(TypeError):
            self.node1.add_neighbour(self.node3,"not_a_number")

    def test_node_comparison(self):
        """Testing the node comparison function"""
        self.assertTrue(self.node1 < self.node2)
        self.assertFalse(self.node1 > self.node2)

    def test_get_index(self):
        """Testing gets the index of the node"""
        self.assertEqual(self.node1.get_index(), 1)

    def test_get_neighbours(self):
        """Testing gets the neighbours of the node"""
        self.node1.add_neighbour(self.node2, 10)
        self.node1.add_neighbour(self.node3, 5)
        expected_neighbours = [(self.node2, 10), (self.node3, 5)]
        actual_neighbours = list(self.node1.neighbours.items())
        self.assertEqual(sorted(actual_neighbours, key=lambda x: x[0].idx),
                         sorted(expected_neighbours, key=lambda x: x[0].idx))

#### Test MinHeap class

In [ ]:
class TestHeap(unittest.TestCase):
    def setUp(self):
        """Set up test fixtures"""
        self.nodes = [Node(i) for i in range(10)]

    def test_init_heap(self):
        """Test initializing heap"""
        heap = MinHeap([(3, self.nodes[3]), (1, self.nodes[1]), (2, self.nodes[2])])
        self.assertEqual(heap.pop()[0],1)
        self.assertEqual(heap.pop()[0],2)
        self.assertEqual(heap.pop()[0],3)

    def test_push_pop(self):
        """Test push and pop to the heap"""
        heap = MinHeap()
        heap.push((2, self.nodes[2]))
        heap.push((1, self.nodes[1]))
        heap.push((3, self.nodes[3]))
        self.assertEqual(heap.pop()[0], 1)
        self.assertEqual(heap.pop()[0], 2)
        self.assertEqual(heap.pop()[0], 3)

    def test_heap_property(self):
        """Test heap property"""
        heap = MinHeap()
        values = [1,2,3,4,5,6,7,8,9]
        for value in values:
            heap.push((value, Node(value)))
        result = []
        while len(heap) > 0:
            result.append(heap.pop()[0])
        self.assertEqual(result, sorted(values))

    def test_top(self):
        """Test the top element of the heap"""
        heap = MinHeap([(2, self.nodes[2]), (1, self.nodes[1]), (3, self.nodes[3])])
        self.assertEqual(heap.top()[0], 1)

    def test_empty_heap(self):
        """Test empty heap"""
        heap = MinHeap()
        self.assertIsNone(heap.top())
        self.assertIsNone(heap.pop())

    def test_duplicate(self):
        """Test duplicates into heap"""
        heap = MinHeap([(2, self.nodes[0]),(2, self.nodes[1]), (2, self.nodes[2])])
        heap.push((2, self.nodes[4]))
        heap.push((1, self.nodes[3]))
        self.assertEqual(heap.top()[0],1)
        heap.pop()
        for _ in range(4):
            self.assertEqual(heap.top()[0],2)
            heap.pop()

    def test_mixed_heap(self):
        """Test mixed situation"""
        heap = MinHeap()
        values = [(2, self.nodes[2]), (1, self.nodes[1]), (3, self.nodes[3])]
        for val in values:
            heap.push(val)
        self.assertEqual(heap.pop()[0],1)
        heap.push((0, self.nodes[0]))
        self.assertEqual(heap.pop()[0],0)

#### Test RandomisedSet class

In [ ]:
class TestRandomisedSet(unittest.TestCase):
    def setUp(self):
        """INITIALIZE a RandomisedSet object"""
        self.randomised_set = RandomisedSet()
        random.seed(0)

    def test_initialization(self):
        """The edges and edge_to_idx of the RandomisedSet should be empty"""
        self.assertFalse(self.randomised_set.edges)
        self.assertFalse(self.randomised_set.edge_to_idx)

    def test_add_edge(self):
        """Add an edge to the RandomisedSet"""
        self.randomised_set.add_edge_to_set(1, 2)
        self.assertEqual(self.randomised_set.edges, [(1, 2)])
        self.assertEqual(self.randomised_set.edge_to_idx, {(1, 2): 0})

    def test_remove_edge(self):
        """Remove an edge from the RandomisedSet"""
        self.randomised_set.add_edge_to_set(1, 2)
        self.randomised_set.remove_edge_from_set(1, 2)
        self.assertNotIn((1, 2),self.randomised_set.edge_to_idx)
        self.assertFalse(self.randomised_set.edges)

    def test_add_duplicate_edges(self):
        """Can not add duplicate edges to the RandomisedSet"""
        self.randomised_set.add_edge_to_set(1, 2)
        self.randomised_set.add_edge_to_set(2, 1)
        self.assertEqual(len(self.randomised_set.edges), 1)

    def test_random_edge(self):
        """Random edge extraction should correctly remove the edge from the set."""
        edges_to_add = [(1, 2), (3, 4)]
        for edge in edges_to_add:
            self.randomised_set.add_edge_to_set(*edge)
        extracted_edges = self.randomised_set.get_random_edge()
        self.assertIn(extracted_edges, edges_to_add)
        self.assertEqual(len(self.randomised_set.edges), 1)
        self.assertNotIn(extracted_edges, self.randomised_set.edge_to_idx)

#### Test Trie class

In [ ]:
class TestTrie(unittest.TestCase):
    def setUp(self):
        self.trie = Trie()
        words = ["apple", "hello", "goodbye", "warwick", "warw", "warwi", "warwic"]
        for word in words:
            self.trie.insert(word)

    def test_insert_and_find(self):
        """Test insert and find"""
        self.assertTrue(self.trie.find("apple"))
        self.assertFalse(self.trie.find("UK"))

    def test_complete(self):
        """Test the complete_function"""
        self.assertIn('warwick', self.trie.complete('warw'))
        self.assertNotIn('apple', self.trie.complete('warw'))

    def test_fizzy_search(self):
        """Test the fizzy_search function"""
        self.assertIn('warwi', self.trie.fizzy_search('warwi', 1))
        self.assertIn('warwic', self.trie.fizzy_search('warwi', 1))
        self.assertNotIn('warwick', self.trie.fizzy_search('warwi', 1))

#### Test Graph class

In [ ]:
class TestGraph(unittest.TestCase):
    def setUp(self):
        """initialize graph object"""
        self.graph = Graph(init_num_nodes=10, add_num_edges=5, edge_sd=5, edge_mean=10)

    def test_graph_initialization(self):
        """Test graph initialization"""
        self.assertEqual(self.graph.num_nodes, 10)
        self.assertGreaterEqual(self.graph.num_edges, 9)

    def test_add_edge(self):
        """Test add edge and validate graph"""
        initial_edges = self.graph.num_edges
        self.graph.add_edge_to_graph(1, 3)
        self.assertEqual(self.graph.num_edges, initial_edges + 1)
        self.assertTrue((1, 3) in self.graph.G.edges() or (2, 3) in self.graph.G.edges())

    def test_add_edge_invalid_edge(self):
        """Test add edge with invalid"""
        with self.assertRaises(ValueError):
            self.graph.add_edge_to_graph(100, 200)

    def test_shortest_path(self):
        """Test shortest path algorithm"""
        path = self.graph.shortest_path(1, 2)
        self.assertIsInstance(path, int)
        self.assertGreater(path, 0)

    def test_random_nodes_edge(self):
        """Test random nodes and edges generator"""
        self.graph.generate_random_nodes(num=2)
        self.assertEqual(self.graph.num_nodes, 12)
        self.graph.generate_random_edges(num=1)
        self.assertGreaterEqual(self.graph.num_edges, 10)

    def test_graph_str_repr(self):
        """Test graph string representation"""
        graph_str = str(self.graph)
        self.assertIn('|E| =', graph_str)
        self.assertIn('|V| =', graph_str)
        self.assertIn('E.x̄ =', graph_str)
        self.assertIn('E.σ =', graph_str)

### Test Game Logic

#### Test RandomScoreGenerator class

In [ ]:
class TestRandomScoreGenerator(unittest.TestCase):
    def setUp(self):
        """set up the test case"""
        self.generator = RandomScoreGenerator()

    def test_init(self):
        """Test the __init__ method."""
        self.assertEqual(self.generator.base_score, 100)
        self.assertIsNone(self.generator._mean)
        self.assertIsNone(self.generator._sd)
        self.assertIsNone(self.generator._generated_distance)

    def test_set_mean_raises_value_error(self):
        """Test set_mean method raises a ValueError"""
        with self.assertRaises(ValueError):
            self.generator.set_mean(-1)
        with self.assertRaises(ValueError):
            self.generator.set_mean(1.1)

    def test_set_mean(self):
        """Test set_mean"""
        self.generator.set_mean(1)
        self.assertEqual(self.generator._mean, 1)

    def test_set_sd_raises_value_error(self):
        """Test set_sd method raises a ValueError"""
        with self.assertRaises(ValueError):
            self.generator.set_sd(-1)
        with self.assertRaises(ValueError):
            self.generator.set_sd("10")

    def test_set_sd(self):
        """Test set_sd correctly sets the sd"""
        self.generator.set_sd(5.0)
        self.assertEqual(self.generator._sd, 5.0)

    @patch('numpy.random.normal')
    def test_generate_random_distance(self, mock_normal):
        """Test generate_random_distance"""
        mock_normal.return_value = 5
        self.generator.set_mean(10)
        self.generator.set_sd(2)
        distance = self.generator.generate_random_distance()
        self.assertEqual(distance, 5)

    def test_generate_random_distance_raises_value_error(self):
        """Test generate_random_distance raises ValueError"""
        with self.assertRaises(ValueError):
            self.generator.generate_random_distance()
        self.generator.set_mean(10)
        with self.assertRaises(ValueError):
            self.generator.generate_random_distance()

    @patch('numpy.random.normal')
    def test_calculate_score(self, mock_normal):
        """Test calculate_score calculates correctly."""
        mean = 10
        sd = 5
        self.generator.set_mean(mean)
        self.generator.set_sd(sd)
        mock_normal.return_value = 10
        '''calculate score'''
        dist = 20
        expected_score = self.generator.calculate_score(dist)
        '''check if score is correct'''
        norm_distribution = stats.norm(mean, sd)
        prob = 1 - norm_distribution.cdf(dist)
        manual_score = int(1 / prob * sd * 2 + self.generator.base_score)
        self.assertEqual(expected_score, manual_score)

    @patch('numpy.random.normal')
    def test_generate_random_edge(self, mock_normal):
        """Test generate_random_edge returns weights"""
        mock_normal.return_value = 5
        weight = RandomScoreGenerator.generate_random_edge(10, 2)
        self.assertEqual(weight, 5)

    def test_generate_random_edge_raises_value_error(self):
        """Test generate_random_edge raises ValueError"""
        with self.assertRaises(ValueError):
            RandomScoreGenerator.generate_random_edge(-10, 2)
        with self.assertRaises(ValueError):
            RandomScoreGenerator.generate_random_edge(10, -2)
        with self.assertRaises(ValueError):
            RandomScoreGenerator.generate_random_edge(10, "2")

#### Test GraphGame class

In [ ]:
class TestGraphGame(unittest.TestCase):
    def setUp(self):
        """Setup for graph"""
        self.game = GraphGame.random_start()
        self.game.set_base_score(100)
        self.game.set_starting_node(1)
        self.game.set_ending_node(4)
        self.game.score_generator = RandomScoreGenerator(self.game.base_score)  # Ensure a generator is set

    def test_set_base_score(self):
        """Test the base score."""
        self.assertEqual(self.game.base_score, 100)
        with self.assertRaises(TypeError):
            self.game.set_base_score('one hundred')

    def test_nodes_existence(self):
        """Test if starting and ending nodes exist in the graph."""
        self.assertIn(self.game.starting_node, self.game.get_nodes())
        self.assertIn(self.game.ending_node, self.game.get_nodes())
        with self.assertRaises(ValueError):
            self.game.set_starting_node(100)  # Assuming node index 100 does not exist

    def test_game_outcome(self):
        """Test the game outcome method."""
        self.game.generate_cutoff()  # First generate the cutoff
        self.assertTrue(isinstance(self.game.check_player_wins(), bool))  # Check if the result is a boolean

    def test_player_score(self):
        """Test score calculation."""
        self.game.generate_cutoff()  # Ensure cutoff is generated
        score = self.game.get_player_score()
        self.assertIsInstance(score, int)  # The score should be an integer

    def test_random_start(self):
        """Test starting a random game."""
        random_game = GraphGame.random_start()
        self.assertIsInstance(random_game, GraphGame)

In [ ]:
unittest.main(argv=[''], verbosity=2, exit=False)

## 4. Start the app

In [ ]:
app = GraphGameGUI()
app.mainloop()